In [81]:
# Read in data
import pandas as pd

stormes = pd.read_csv('./../Data/Stormdata_2006.csv', encoding='iso-8859-1')
queries= pd.read_csv('./../Data/QueriesToYT.csv', encoding='iso-8859-1')

In [82]:
stormes['BEGIN_DATE_TIME']

0           1/1/2006 0:00
1           1/1/2006 0:00
2           1/1/2006 0:00
3           1/1/2006 0:00
4           1/1/2006 0:00
               ...       
48590    10/16/2006 14:18
48591    10/16/2006 15:30
48592    10/16/2006 15:30
48593    10/16/2006 16:45
48594    10/27/2006 15:00
Name: BEGIN_DATE_TIME, Length: 48595, dtype: object

In [83]:
stormes.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'LAST_MOD_DATE',
       'LAST_MOD_TIME', 'LAST_CERT_DATE', 'LAST_CERT_TIME', 'LAST_MOD',
       'LAST_CERT', 'ADDCORR_FLG', 'ADDCORR_DATE'],
      dtype='object')

In [84]:
queries.columns

Index(['Unnamed: 0', 'QUERY', 'month', 'day of the month', 'DAYSAFTERDBSTART',
       'hour', 'minute', 'QUERYID', 'TIMEID', 'ANONID',
       'MINUTESAFTERMIDNIGHT'],
      dtype='object')

In [85]:
stormes.head(3)

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,EPISODE_NARRATIVE,EVENT_NARRATIVE,LAST_MOD_DATE,LAST_MOD_TIME,LAST_CERT_DATE,LAST_CERT_TIME,LAST_MOD,LAST_CERT,ADDCORR_FLG,ADDCORR_DATE
0,200601,1,0,200601,31,2359,202408,5482479,COLORADO,8,...,The storm track favored northwest Colorado wit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200601,1,0,200601,31,2359,203632,5486559,TEXAS,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200601,1,0,200601,31,2359,203632,5486558,TEXAS,48,...,NaN,Drought conditions intensified across all of S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
queries.head(3)

,Unnamed: 0,QUERY,month,day of the month,DAYSAFTERDBSTART,hour,minute,QUERYID,TIMEID,ANONID,MINUTESAFTERMIDNIGHT
0,0,jake peyton,march,1,0,0,13,2775687,5962961,154767,13
1,1,bush drinking video,april,26,56,9,31,2778308,5986392,53322,571
2,2,cant stop scratching my vagina,april,26,56,2,19,2774391,4104872,317691,139


In [87]:
stormes['YEAR']

0        2006
1        2006
2        2006
3        2006
4        2006
         ... 
48590    2006
48591    2006
48592    2006
48593    2006
48594    2006
Name: YEAR, Length: 48595, dtype: int64

In [88]:
stormes['MONTH_NAME']

0        January  
1        January  
2        January  
3        January  
4        January  
           ...    
48590    October  
48591    October  
48592    October  
48593    October  
48594    October  
Name: MONTH_NAME, Length: 48595, dtype: object

In [89]:
stormes['END_DATE_TIME'] = pd.to_datetime(stormes['END_DATE_TIME'])
stormes['BEGIN_DATE_TIME'] = pd.to_datetime(stormes['BEGIN_DATE_TIME'])

In [90]:
import pandas as pd
from datetime import datetime

# Convert month names to their numeric equivalent
queries['month'] = queries['month'].str.strip().str.capitalize()

# Convert month names to their numeric equivalent, ensuring 2-digit format
queries['month_numeric'] = pd.to_datetime(queries['month'], format='%B').dt.month.astype(str).str.zfill(2)
# Build the datetime string with the correct numeric month
queries['QUERY_DATETIME'] = pd.to_datetime(
    '2006-' + queries['month_numeric'].astype(str).str.zfill(2) + '-' + 
    queries['day of the month'].astype(str).str.zfill(2) + ' ' +
    queries['hour'].astype(str).str.zfill(2) + ':' + 
    queries['minute'].astype(str).str.zfill(2) + ':00',
    format="%Y-%m-%d %H:%M:%S"
)


In [93]:
# Define the function to join based on the datetime range condition
def join_on_date_range(stormes, queries):
    result = []
    
    for _, query in queries.iterrows():
        query_time = query['QUERY_DATETIME']
        
        # Filter stormes where BEGIN_DATE_TIME <= QUERY_DATETIME <= END_DATE_TIME
        matched_stormes = stormes[
            (stormes['BEGIN_DATE_TIME'] <= query_time) & 
            (stormes['END_DATE_TIME'] >= query_time)
        ]
        
        # If there are matches, add the QUERY column to the matched rows
        if not matched_stormes.empty:
            matched_stormes_copy = matched_stormes.copy()  # Create a copy
            matched_stormes_copy['QUERY'] = query['QUERY']  # Add the QUERY column
            result.append(matched_stormes_copy)
    
    # Concatenate the list of matched results into a single dataframe
    if result:
        return pd.concat(result, ignore_index=True)
    else:
        return pd.DataFrame()  # Return empty dataframe if no matches

# Perform the join
joined_df = join_on_date_range(stormes, queries)


In [102]:
def left_join_on_date_range(stormes, queries):
    result = []

    # Iterate over each query
    for _, query in queries.iterrows():
        query_time = query['QUERY_DATETIME']

        # Filter stormes where BEGIN_DATE_TIME <= QUERY_DATETIME <= END_DATE_TIME
        matched_stormes = stormes[
            (stormes['BEGIN_DATE_TIME'] <= query_time) &
            (stormes['END_DATE_TIME'] >= query_time)
        ]

        # If there are matches, add the QUERY column to the matched storm rows
        if not matched_stormes.empty:
            matched_stormes_copy = matched_stormes.copy()  # Create a copy to avoid modification
            matched_stormes_copy['QUERY'] = query['QUERY']  # Add the QUERY column
            result.append(matched_stormes_copy)
        else:
            # If no storm matches, keep the query row with NaN values for the storm data
            empty_storm_row = pd.DataFrame([query], columns=queries.columns)
            empty_storm_row = empty_storm_row.assign(**{col: None for col in stormes.columns})
            empty_storm_row['QUERY'] = query['QUERY']  # Assign query column
            result.append(empty_storm_row)

    # Concatenate the list of matched results into a single dataframe
    if result:
        return pd.concat(result, ignore_index=True)
    else:
        return pd.DataFrame()  # Return an empty dataframe if no matches

# Perform the left join
joined_df2 = left_join_on_date_range(stormes, queries)


In [104]:
joined_df2.shape

(499541, 59)

In [95]:
joined_df.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'LAST_MOD_DATE',
       'LAST_MOD_TIME', 'LAST_CERT_DATE', 'LAST_CERT_TIME', 'LAST_MOD',
       'LAST_CERT', 'ADDCORR_FLG', 'ADDCORR_DATE', 'QUERY'],
      dtype='object'

In [107]:
joined_df[['EVENT_TYPE','QUERY','BEGIN_DATE_TIME','END_DATE_TIME']]

,EVENT_TYPE,QUERY,BEGIN_DATE_TIME,END_DATE_TIME
0,Blizzard,jake peyton,2006-03-01 00:00:00,2006-03-01 06:35:00
1,Flood,jake peyton,2006-03-01 00:00:00,2006-03-31 23:59:00
2,Flood,jake peyton,2006-03-01 00:00:00,2006-03-31 23:59:00
3,Flood,jake peyton,2006-03-01 00:00:00,2006-03-31 23:59:00
4,Drought,jake peyton,2006-03-01 00:00:00,2006-03-31 23:59:00
...,...,...,...,...
499536,Heavy Snow,star wars cops,2006-03-12 22:30:00,2006-03-14 05:30:00
499537,Heavy Snow,star wars cops,2006-03-12 22:30:00,2006-03-14 05:30:00
499538,Wildfire,star wars cops,2006-03-12 23:00:00,2006-03-17 18:00:00
499539,Flash Flood,star wars cops,2006-03-13 00:00:00,2006-03-13 02:00:00


In [100]:
joined_df.shape

(499541, 59)

In [101]:
queries.shape

(2028, 13)

## Questions:
- How do search queries related to extreme weather events change before, during, and after a major weather event?
- How do search behaviors differ by region during extreme weather events?
- What keywords do users search for when a specific type of weather event occurs, such as a tornado vs. a hurricane?
- Do users tend to search more for weather-related information in preparation for events, or are they primarily reactive during and after the event?
- How does media coverage of extreme weather events influence search behavior?

In [115]:
stormes[stormes['EVENT_TYPE']=="Tornado"]["BEGIN_DATE_TIME"]

443     2006-01-01 15:25:00
503     2006-01-02 03:00:00
507     2006-01-02 04:45:00
514     2006-01-02 05:25:00
529     2006-01-02 08:02:00
                ...        
48475   2006-09-28 17:34:00
48517   2006-09-28 19:35:00
48521   2006-09-29 01:37:00
48533   2006-09-29 13:54:00
48555   2006-09-30 16:32:00
Name: BEGIN_DATE_TIME, Length: 1093, dtype: datetime64[ns]

In [116]:
stormes[stormes['EVENT_TYPE']=="Tornado"]["END_DATE_TIME"]

443     2006-01-01 15:27:00
503     2006-01-02 03:01:00
507     2006-01-02 04:46:00
514     2006-01-02 05:27:00
529     2006-01-02 08:03:00
                ...        
48475   2006-09-28 17:44:00
48517   2006-09-28 19:40:00
48521   2006-09-29 01:37:00
48533   2006-09-29 13:54:00
48555   2006-09-30 16:32:00
Name: END_DATE_TIME, Length: 1093, dtype: datetime64[ns]

In [122]:
queries["QUERY_DATETIME"]

0      2006-03-01 00:13:00
1      2006-04-26 09:31:00
2      2006-04-26 02:19:00
3      2006-04-19 16:06:00
4      2006-05-09 15:44:00
               ...        
2023   2006-05-24 11:40:00
2024   2006-05-25 20:49:00
2025   2006-03-04 13:40:00
2026   2006-03-04 13:40:00
2027   2006-03-13 01:10:00
Name: QUERY_DATETIME, Length: 2028, dtype: datetime64[ns]

In [127]:
stormes = stormes[stormes['BEGIN_DATE_TIME'] >= '2006-03-01 00:00:00']
stormes = stormes[stormes['END_DATE_TIME'] <= '2006-06-01 00:00:00']

In [138]:
result=[]
for start,end in zip(stormes[stormes['EVENT_TYPE']=="Tornado"]["BEGIN_DATE_TIME"],stormes[stormes['EVENT_TYPE']=="Tornado"]["END_DATE_TIME"]):
    queries_before=queries[queries["QUERY_DATETIME"]<start]
    queries_during=queries[(queries["QUERY_DATETIME"]>=start) & (queries["QUERY_DATETIME"]<=end)]
    queries_after=queries[queries["QUERY_DATETIME"]>end]
    before_count = queries_before.shape[0]
    after_count = queries_after.shape[0]
    during_count = queries_during.shape[0]
    result.append({
            'Tornado_ID': 'Tornado',  # Assuming 'EVENT_ID' is the unique identifier for each event
            'Before_Count': before_count,
            'After_Count': after_count,
            'During_Count': during_count,
            'Before_Queries': queries_before['QUERY'].tolist(),  # List of queries before
            'After_Queries': queries_after['QUERY'].tolist(),   # List of queries after
            'During_Queries': queries_during['QUERY'].tolist()
        })
result_df=pd.DataFrame(result)

In [139]:
result_df

,Tornado_ID,Before_Count,After_Count,During_Count,Before_Queries,After_Queries,During_Queries
0,Tornado,179,1849,0,"[jake peyton, elkins high school, slap happy, ...","[bush drinking video, cant stop scratching my ...",[]
1,Tornado,179,1849,0,"[jake peyton, elkins high school, slap happy, ...","[bush drinking video, cant stop scratching my ...",[]
2,Tornado,179,1849,0,"[jake peyton, elkins high school, slap happy, ...","[bush drinking video, cant stop scratching my ...",[]
3,Tornado,179,1849,0,"[jake peyton, elkins high school, slap happy, ...","[bush drinking video, cant stop scratching my ...",[]
4,Tornado,179,1849,0,"[jake peyton, elkins high school, slap happy, ...","[bush drinking video, cant stop scratching my ...",[]
...,...,...,...,...,...,...,...
635,Tornado,2019,9,0,"[jake peyton, bush drinking video, cant stop s...","[blood gang, adult swim home movies, dave chap...",[]
636,Tornado,2019,9,0,"[jake peyton, bush drinking video, cant stop s...","[blood gang, adult swim home movies, dave chap...",[]
637,Tornado,2019,9,0,"[jake peyton, bush drinking video, cant stop s...","[blood gang, adult swim home movies, dave chap...",[]
638,Tornado,2019,9,0,"[jake peyton, bush drinking video, cant stop s...","[blood gang, adult swim home movies, dave chap...",[]
